In [1]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from util import datasets, methods, models
from benchmark.sensitivity_n import sensitivity_n
import itertools
import numpy as np

In [6]:
dataset = datasets.Cifar(batch_size=16, data_location="../data/CIFAR10", download=False, shuffle=False, version="cifar10")
model = models.CifarResnet(version="resnet32", params_loc="../data/models/cifar10_resnet32.pth", num_classes=10, output_logits=False)

KeyboardInterrupt: 

In [ ]:
DATASET = "MNIST"
DOWNLOAD_DATASET = False
MODEL = "CNN"
BATCH_SIZE = 64
N_BATCHES = 2
N_SUBSETS = 100
#MASK_RANGE = range(1, 1000, 100)
MASK_RANGE = range(1, 700, 50)
METHODS = ["GuidedGradCAM", "Gradient", "InputXGradient", "IntegratedGradients",
           "GuidedBackprop", "Deconvolution", "Random"]

# TODO instead of having this global dictionary, expose getter methods that do the necessary validations
dataset_constructor = DATASET_MODELS[DATASET]["constructor"]
model_constructor = DATASET_MODELS[DATASET]["models"][MODEL]
method_constructors = get_method_constructors(METHODS)

all_kwargs = {"Occlusion": {"sliding_window_shapes": (1, 1, 1)}}

model = model_constructor(output_logits=True)
dataset = dataset_constructor(batch_size=BATCH_SIZE, shuffle=False, download=DOWNLOAD_DATASET,
                              data_location="../../data")

x = np.array(MASK_RANGE)
methods = {m_name: method_constructors[m_name](model, normalize=True, **all_kwargs.get(m_name, {})) for m_name in METHODS}
sensitivity_n(itertools.islice(dataset.get_test_loader(), N_BATCHES), lambda x: model.predict(x).detach().numpy(),
              methods, list(MASK_RANGE), mask_value=-0.4242)